In [1]:
import pandas as pd
import urllib.parse
from selenium import webdriver
from bs4 import BeautifulSoup

# CSV 파일에서 데이터 읽어오기
data = pd.read_csv('Nowon.csv', encoding='cp949')

# 음식과 숙박을 제외한 행만 필터링
filtered_data = data[(data['KTO카테고리대분류명'] != '음식') & (data['KTO카테고리대분류명'] != '숙박')]

# 관광지명 추출
tourist_spots = filtered_data['관광지명'].tolist()

# 중복 관광지명 제거
tourist_spots = list(set(tourist_spots))

# Chrome WebDriver 초기화
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창이 나타나지 않도록 설정
driver = webdriver.Chrome(options=options)

# 결과를 저장할 데이터 프레임 생성
result_df = pd.DataFrame(columns=['관광지명', '숙박시설명', 'URL'])

# 관광지명에 대해 반복
for spot in tourist_spots:
    # Google 지도 페이지로 이동
    url = f"https://www.google.com/maps/search/{urllib.parse.quote(spot)}+%EC%88%99%EB%B0%95%EC%8B%9C%EC%84%A4/data=!3m1!4b1?hl=ko&entry=ttu"
    driver.get(url)
    
    # 페이지의 HTML 가져오기
    page_content = driver.page_source
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # 클래스가 'hfpxzc'인 모든 요소(숙박시설) 찾기
    elements = soup.find_all('a', class_='hfpxzc')
    
    # 관광지별 숙박시설 목록을 저장할 리스트 초기화
    accommodations = []
    
    # 결과를 데이터프레임에 추가
    for element in elements:
        aria_label = element['aria-label']
        href = element['href']
        
        # 중복 숙박시설 제거
        if aria_label not in accommodations:
            accommodations.append(aria_label)
            # 결과 데이터프레임에 추가
            result_df = pd.concat([result_df, pd.DataFrame({'관광지명': [spot], '숙박시설명': [aria_label], 'URL': [href]})], ignore_index=True)
    
    # 관광지별로 숙박시설명이 안 나오는 경우 해당 관광지 삭제
    if len(accommodations) == 0:
        result_df = result_df[result_df['관광지명'] != spot]
        
# 숙박시설명과 관광지명이 겹치는 경우 해당 행 삭제
result_df = result_df[result_df['숙박시설명'] != result_df['관광지명']]

# 크롤링이 끝나면 WebDriver 종료
driver.quit()

# 결과 데이터프레임 출력
print(result_df)

         관광지명     숙박시설명                                                URL
0    화랑대 철도공원       W호텔  https://www.google.com/maps/place/W%ED%98%B8%E...
1    화랑대 철도공원  갈매 퍼스트호텔  https://www.google.com/maps/place/%EA%B0%88%EB...
2    화랑대 철도공원       르블랑  https://www.google.com/maps/place/%EB%A5%B4%EB...
3    화랑대 철도공원       S호텔  https://www.google.com/maps/place/S%ED%98%B8%E...
4    화랑대 철도공원      태릉 인  https://www.google.com/maps/place/%ED%83%9C%EB...
..        ...       ...                                                ...
154      화랑대역       W호텔  https://www.google.com/maps/place/W%ED%98%B8%E...
155      화랑대역       르블랑  https://www.google.com/maps/place/%EB%A5%B4%EB...
156      화랑대역      태릉 인  https://www.google.com/maps/place/%ED%83%9C%EB...
157      화랑대역       S호텔  https://www.google.com/maps/place/S%ED%98%B8%E...
158      화랑대역      QT모텔  https://www.google.com/maps/place/QT%EB%AA%A8%...

[157 rows x 3 columns]


In [2]:
# 결과 데이터프레임을 CSV 파일로 저장
result_df.to_csv('Nowon_stay.csv', index=False, encoding='utf-8-sig')